# Sensitivity analysis 

using OGGM edu glacier model

## Acknowledgement

by Thomas Gölles, 2024

In [ ]:
from oggm_edu import GlacierBed, Glacier, MassBalance, GlacierCollection
import SALib
from SALib.sample.sobol import sample
from SALib.analyze.sobol import analyze
import numpy as np

from matplotlib import pyplot as plt

print("salib version: ", SALib.__version__)

## General OGGM definitions

In [ ]:
# Lets define the bed.
# Top and bottom altitude.
top = 5000
bottom = 0
# How far does the accumulation area extend?
accumulation_fraction = 1 / 3
# ELA altitude
ela_alt = (top - bottom) * (1 - accumulation_fraction)
# Accumulation area starts out as 1000 m. wide.
top_width = 1000
# 500 m. wide after ELA.
ela_width = 500

# Bed, note that we increase the map_dx here,
# we increase the grid point spacing.
bed = GlacierBed(
    altitudes=[top, ela_alt, bottom],
    widths=[top_width, ela_width, ela_width],
    map_dx=200,
)

Lets investigate how the mass blance influences the volume

In [ ]:
mb_gradient = 7
mass_balance = MassBalance(ela=ela_alt, gradient=mb_gradient)

In [ ]:
glacier = Glacier(bed, mass_balance=mass_balance)
glacier.progress_to_equilibrium()

In [ ]:
glacier

In [ ]:
glacier.plot()

In [ ]:
glacier.plot_history()

In [ ]:
glacier.history

We need a function which takes the mb_graident and gives us the resulting glacier in quilibrium. Then we varay the massbalance gradient and analyse the outcome.

In [ ]:
def evaluate_model(mb_gradient):
    mass_balance = MassBalance(ela=ela_alt, gradient=mb_gradient)
    glacier = Glacier(bed, mass_balance=mass_balance)
    glacier.progress_to_equilibrium()
    return glacier

## OAT sensitivity analysis

We interessed in the equilibirum glacier volume. And we want to know how the glacier reacts to massbalance gadients form 0.01 to 10.

In [ ]:
mb_gradient_min = 0.01
mb_gradient_max = 10
number_of_samples = 100
np.random.seed(0)  # so the results are reproducible and everyone gets the same results
mb_gradient_samples = np.random.uniform(
    mb_gradient_min, mb_gradient_max, number_of_samples
)
mb_gradient_samples

Ok, now we have the sample for our one one at the time sensitivity analysis.

Now lets generate a list with all the glaciers for the different mass balance gradients. We need an empty list to which we append the results run in a for loop:

In [ ]:
results = []
for mb_gradient in mb_gradient_samples:
    glacier = evaluate_model(mb_gradient)
    results.append(glacier)
    print(f"Done with mb_gradient: {mb_gradient}")

Now OGGM_edu has a usefull class called GlacierCollection

In [ ]:
ota_runs = GlacierCollection(results)

In [ ]:
# plot mass balances
fig, ax = plt.subplots()
for glacier in ota_runs.glaciers:
    ax.plot(
        glacier.annual_mass_balance,
        glacier.bed.bed_h,
        label=f"Glacier {glacier.id}, " + f"gradient {glacier.mass_balance.gradient}",
    )
    # Add each ELA.
    # elas.append(glacier.mass_balance.ela)
    # Add labels.
    ax.set_xlabel("Annual mass balance [m yr-1]")
    ax.set_ylabel("Altitude [m]")

Now lets plot the equilibrium volume vs the mass balance gradient:

In [ ]:
plt.scatter(mb_gradient_samples, ota_runs.summary()["Volume [km3]"])
plt.xlabel("Mass balance gradient")
plt.ylabel("Volume [km3]")

What do we see here?

* The volume depends on the mass balance gradient
* it is non-linear
* it looks exponential
* where there is a lot of change at low mass balance gradients we have rader few points

What are soem real world implications*

* Glaciers with low mass balance gradient are more sensitive to climate

Limitations of OAT:

* no interactions between multiple factors 
* in reality multiply factors change at teh same time

OAT is usefull for inital insights but to capture feedbacks more advanced sensitivity analysis methods are needed.

## Using SALib and the Sobol method with one varible

Lets repeat the same experiment but using SALib.
First we need to defien the "priboblem", i.e, the epxeriment 
Again vary the mb gradient form 0.01 to 10

In [ ]:
problem = {"num_vars": 1, "names": ["mb_gradient"], "bounds": [[0.01, 10]]}

Instead of choosing randomly we use the Sobol sampler

In [ ]:
number_of_samples = 50

Now lets generate the samples. i.e. the list of inputs to test the model with

In [ ]:
param_values = sample(problem, number_of_samples)

In [ ]:
param_values

we reuse the function "elevate_model" from above. Which takes the mb_gradient as input and runs the glacier to equilibrium.
We need to adjust the for loop a bit since, the param_values with our sampel is a list of lists. (therfore the forcing[0])



In [ ]:
results = []
for forcing in param_values:
    mb_gradient = forcing[0]
    glacier = evaluate_model(forcing)
    results.append(glacier)
    print(f"Done with mb_gradient: {mb_gradient}")

In [ ]:
sobol_runs = GlacierCollection(results)

In [ ]:
plt.scatter(param_values, sobol_runs.summary()["Volume [km3]"])
plt.xlabel("Mass balance gradient")
plt.ylabel("Volume [km3]")

What to we see here?

* more evently spaced samples
* the same conclustions as with the somple OAT approach.

Now we can actually analyse the Sobol Sensitivity measures:

In [ ]:
sobol_result = analyze(problem, sobol_runs.summary()["Volume [km3]"].values)

In [ ]:
sobol_result

What do the values mean?

* S1 is the first-order sensitivity index and should be between 0 and 1
* S1 is larger than 1 => might need more samples. In this case it should be 1 since all the Variance of the output is form one input (the mass balance gradient)
* S1_conf quite large so significant uncertainties about the S1

* ST the Total Sensitivity Index sould also be 1 and the same is S1 since ther are not interactions with other variables

* S2 has a  not a number value (nan), since we only had one paramter.

So sobol sensitivity for one varialbe gives not much more insight then OTA, but now lets try it for multiple inputs


## Sobol sensitivity for multiple inputs

In [ ]:
def evaluate_model_multi(mb_gradient, ela_width, add_to_with_at_top):
    top = 5000
    bottom = 0

    accumulation_fraction = 1 / 3
    ela_alt = (top - bottom) * (1 - accumulation_fraction)
    top_width = ela_width + add_to_with_at_top
    bed = GlacierBed(
        altitudes=[top, ela_alt, bottom],
        widths=[top_width, ela_width, ela_width],
        map_dx=200,
    )
    mass_balance = MassBalance(ela=ela_alt, gradient=mb_gradient)
    glacier = Glacier(bed, mass_balance=mass_balance)
    glacier.progress_to_equilibrium()
    return glacier

In [ ]:
problem = {
    "num_vars": 3,
    "names": ["mb_gradient", "ela_width", "add_to_with_at_top"],
    "bounds": [
        [0.01, 10],
        [500, 2000],
        [0, 1000],
    ],
}

In [ ]:
number_of_samples = 50

In [ ]:
param_values = sample(problem, number_of_samples)

In [ ]:
results = []
for forcing in param_values:
    glacier = evaluate_model_multi(
        mb_gradient=forcing[0], ela_width=forcing[1], add_to_with_at_top=forcing[2]
    )
    results.append(glacier)
    print(f"Done with forcings: {forcing}")

In [ ]:
sobol_results_multi = GlacierCollection(results)

In [ ]:
sobol_results_multi

Lets now blot the mass balance gradient and volume

In [ ]:
mb_gradients = param_values[:, 0]
plt.scatter(mb_gradients, sobol_results_multi.summary()["Volume [km3]"])
plt.xlabel("Mass balance gradient")
plt.ylabel("Volume [km3]")

What do we see here?

* there is much mor variation in total
* not just depending on the Mass balance gradient
* The OTA relation ship is still visible

Lets look at the other values

In [ ]:
ela_widths = param_values[:, 1]
plt.scatter(ela_widths, sobol_results_multi.summary()["Volume [km3]"])
plt.xlabel("ELA width")
plt.ylabel("Volume [km3]")

* looks like a linear relationship and interactions with other iputs
* not supprising the wider the glacier is at the larger it is, but it is not the only relevant intput

In [ ]:
add_to_with_at_tops = param_values[:, 2]
plt.scatter(add_to_with_at_tops, sobol_results_multi.summary()["Volume [km3]"])
plt.xlabel("add_to_with_at_tops [m]")
plt.ylabel("Volume [km3]")

* All over the place, looks like no stron correlation

In [ ]:
sobol_multi = analyze(problem, sobol_results_multi.summary()["Volume [km3]"].values)

In [ ]:
total_Si, first_Si, second_Si = sobol_multi.to_df()

In [ ]:
total_Si

In [ ]:
total_Si.sum()

Total Effect:

So the largest total effect comes form the ELA with, followed by the mb_gradient.
the added glacier witha the top as a rather small effect.
Although note theat eh confidence intervall is rather larger, which ist moste likely du to the small sample size.

In [ ]:
first_Si

S1 effect:

The S1 index represents the direct effect of each input parameter on the output variance, without considering any interaction effects.
The ranking is the same as for the ST

In [ ]:
second_Si

S2 effect

Mass balance gradient and add with to topa interact the most. Likely toto a larger accumulation area when the with at the top is larger and therfore the mass balance gradient has a larger effect.
The S2 conf values are quite high, so the conclusions need to be taken with a caution. A larger number of samples would be a good start to investigate this futher and see if it reduces the S2_conf values.

In [ ]:
sobol_multi.plot()

In [ ]:
sobol_results_multi.glaciers[150].plot()